In [1]:
from resnet_variant import *
from baseline_cnn import *
from resnet_variant import resnet_n
from resnet import resnet50

import torch.nn as nn
import torch.optim as optim
import time
import pathlib
import torch
from evaluation import Evaluation

%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.00001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

class channelCopy(object):    
    def __call__(self, img):
        return torch.cat([img, img, img], 0)
    
def augmentImg(object):    
    ang_rand = np.random.choice(10,1)
    if ang_rand==0:
            

#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),transforms.ToTensor(),channelCopy()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)

CUDA is supported


In [34]:
np.random.choice(10,1)

array([9])

In [6]:
#Select models for evaluation
mode = 'resnet_variant'
if mode == 'resnet_variant':
    PATH = '/datasets/home/home-02/60/960/kshi/PA3/models/resnet_variant/weighted/20190216-131303/epoch_0-batch_1800-loss_5.402564784162678e-05-20190216-142005.pt'
    model_test = resnet_n(pretrained=False, num_classes=14)
    model_test = model_test.to(computing_device)
    model_test.load_state_dict(torch.load(PATH))
elif mode == 'resnet':
    PATH = '/datasets/home/home-02/60/960/kshi/PA3/models/resnet50/20190214-113501/epoch_0-batch_1900-loss_0.17764276266098022-20190214-154931.pt'
    model_test = resnet50(pretrained=False, num_classes=14)
    model_test = model_test.to(computing_device)
    model_test.load_state_dict(torch.load(PATH))

In [7]:
labels_all = []
predictions_all = []
model_test.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(computing_device), labels.to(computing_device)
        labels_all.append(labels)
        output = model_test(images)
        predictions = output > 0.5
        predictions_all.append(predictions)

labels = torch.cat(labels_all,0)
predctions = torch.cat(predictions_all,0)

## resnet

In [5]:
eval = Evaluation(predctions.float(), labels)
eval.evaluate()

TP [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
FP [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
TN [18080.0, 19717.0, 17878.0, 16513.0, 19116.0, 19004.0, 19903.0, 19246.0, 19314.0, 19758.0, 19756.0, 19894.0, 19571.0, 20138.0]
FN [2101.0, 464.0, 2303.0, 3668.0, 1065.0, 1177.0, 278.0, 935.0, 867.0, 423.0, 425.0, 287.0, 610.0, 43.0]
accuracy tensor([0.8959, 0.9770, 0.8859, 0.8182, 0.9472, 0.9417, 0.9862, 0.9537, 0.9570,
        0.9790, 0.9789, 0.9858, 0.9698, 0.9979], device='cuda:0',
       dtype=torch.float64)
precision tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0', dtype=torch.float64)
recall tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', dtype=torch.float64)


## resnet_variant_weighted

In [8]:
eval = Evaluation(predctions.float(), labels)
eval.evaluate()

TP [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
FP [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
TN [18080.0, 19717.0, 17878.0, 16513.0, 19116.0, 19004.0, 19903.0, 19246.0, 19314.0, 19758.0, 19756.0, 19894.0, 19571.0, 20138.0]
FN [2101.0, 464.0, 2303.0, 3668.0, 1065.0, 1177.0, 278.0, 935.0, 867.0, 423.0, 425.0, 287.0, 610.0, 43.0]
accuracy tensor([0.8959, 0.9770, 0.8859, 0.8182, 0.9472, 0.9417, 0.9862, 0.9537, 0.9570,
        0.9790, 0.9789, 0.9858, 0.9698, 0.9979], device='cuda:0',
       dtype=torch.float64)
precision tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0', dtype=torch.float64)
recall tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0', dtype=torch.float64)
